In [1]:
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
import vectordb.chunk_utils as chunk_utils
from vectordb import load_test_data
from vectordb.embedding.embedder import Embedder


file_path = "../data/visindavefur_articles.json"

with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

data[0].keys()

/Users/einar/git/hafsteinn/together_rag/venv/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


dict_keys(['name', 'url', 'main_text'])

In [2]:
chunks = []

for i, article in enumerate(data):
    article_chunks = chunk_utils.create_chunks(
        text=article["main_text"],
        language="is",
        pinecone_index_name="is_index",
        pinecone_namespace="visindavefur",
        source_title=article["name"],
        source_url=article["url"],
        chunk_size=500,
        chunk_overlap=150,
    )
    chunks.extend(article_chunks)
print(len(chunks))
chunks[0]

5


Chunk(id='eae4f489711a23e5590be365474900a1#0', language='is', pinecone_index_name='is_index', pinecone_namespace='visindavefur', chunk_text='Hlusta\n\n\tÖll spurningin hljóðaði svona: \n\n  Í hvaða hæð frá jörðu er vindhraði mældur? Skiptir máli í hvaða hæð hann er mældur?\n\n\nVindhraði er að jafnaði mældur í 10 metra hæð yfir jörðu og er það í samræmi við reglur \nAlþjóðaveðurfræðistofnunar\n Sameinuðu þjóðanna. Það kostar töluvert að koma vindhraðamæli upp í 10 metra hæð og þess vegna mæla margir vindinn nær jörðu. Algengt er til dæmis að vindhraðamælar Vegagerðarinnar séu í 6-7 metra hæð yfir umhverfi mælisins.', chunk_index=0, source_title='náttúruvísindi og verkfræðiÍ hvaða hæð frá jörðu er vindhraði mældur?', source_url='/svar.php?id=78415', vector=None)

## Now embed the chunks


In [3]:
chunk_texts = [chunk.chunk_text for chunk in chunks]
embedder = Embedder()
chunk_vectors = embedder.embed(chunk_texts)

for i, chunk in enumerate(chunks):
    chunk.vector = chunk_vectors[i]

## Now insert the chunks into the sqlite db

In [14]:
import datastore

datastore.delete_chunks([chunk.id for chunk in chunks])
datastore.insert_chunks(chunks)

## now insert the chunks into pinecone

In [9]:
import vectordb.pinecone_interface as pinecone_interface
from importlib import reload

reload(pinecone_interface)

pinecone_interface.create_indices()

pinecone_interface.insert_vectors(chunks, "is-index", "visindavefur")

Index en-index already exists
Index is-index already exists


In [10]:
print(len(data))
print(len(chunks))

20
5
